In [2]:
import pandas as pd
import numpy as np
import igraph

In [13]:
### Loading Raw Data
df = {}
with open('./amzn_reviews/reviews_Health_and_Personal_Care_5.json', 'r') as f:
    for i, line in enumerate(f.readlines()):
        df[i] = eval(line)

raw_data = pd.DataFrame.from_dict(df, orient='index')

# fixing the helpful is a list issue
raw_data['helpful_ct'] = raw_data.helpful.apply(lambda x : x[0])
raw_data['unhelpful_ct'] = raw_data.helpful.apply(lambda x : x[1] - x[0])
raw_data.drop('helpful', axis=1, inplace=True)

# subset most popular products (> 50 reviews)
asin_list = list(raw_data.groupby(['asin']).count().sort_values('reviewerID',ascending=False).query('reviewerID > 50').index)

In [3]:
# saving the file
#raw_data[raw_data['asin'].isin(asin_list)].reset_index().to_feather('./amzn_popular_product_stats.feather')

# reading the file
raw_data = pd.read_feather('./amzn_popular_product_stats.feather')

In [4]:
raw_data.head()

,index,reviewerID,asin,reviewerName,reviewText,overall,summary,unixReviewTime,reviewTime,helpful_ct,unhelpful_ct
0,80,A2BVV8ND8Q6S6A,B00000J47L,adero1780,This was a great purchase at a nice price. If ...,5.0,These batteries are great,1357862400,"01 11, 2013",0,0
1,81,A2P0Z6QPAYNXGM,B00000J47L,Allan Hamilton,No problem. Well pleased.,5.0,Well pleased.,1404864000,"07 9, 2014",0,0
2,82,A2CHN4OYWUBKQX,B00000J47L,Amazon Customer,These are nice to have so I give them a 4 out ...,4.0,Nice to have.,1385596800,"11 28, 2013",0,0
3,83,AKU18EGFF6G02,B00000J47L,Amazon Customer,I bought a pack of these a while back from Sam...,1.0,Expensive junk,1312416000,"08 4, 2011",1,0
4,84,A2E95ZZNAQ24GS,B00000J47L,"Amazon Customer ""mikeklement.com""",Dont use that often but after a couple years o...,5.0,Work Great,1353974400,"11 27, 2012",0,0


## Getting Pairwise Connections per User

In [15]:
import itertools

In [17]:
list(itertools.combinations([1,2,3],2))

[(1, 2), (1, 3), (2, 3)]

In [ ]:
review_to_asin = raw_data.groupby(['reviewerID']).asin.apply(lambda x : set(x))

In [21]:
asin_to_review = raw_data.groupby('asin').reviewerID.apply(lambda x : set(x))

In [25]:
test_ ={prod : list(itertools.combinations(usrs, 2)) for prod,usrs in asin_to_review.items()} 

In [26]:
test_

In [10]:
g = igraph.Graph()